In [5]:
import openai
import os
import json

from typing import List, Dict, Any
from pathlib import Path
from dataclasses import dataclass

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

# print(openai.Engine.list())

global_path: Path = Path("../../lib/StaticDataStorage/data/Global.json").resolve()

def load_global_static_data() -> Dict[str, Any]:
    return json.loads(global_path.read_text())

def get_terms(globalJson: Dict[str, Any]) -> List[str]:
    ret: List[str] = []

    for item in globalJson['terms']:
        termDef = item['termDef']
        term = termDef.split(" - ")[0]
        ret.append(term)

    return ret

def get_terms_string() -> str:
    globalJson = load_global_static_data()
    terms = get_terms(globalJson)
    return ", ".join(terms)

def print_lines(text: str, max_line_length: int = 120):
    lines: List[str] = []
    line = ""
    for word in text.split():
        if len(line + word) > max_line_length:
            lines.append(line)
            line = ""
        line += word + " "
    lines.append(line)

    for line in lines:
        print(line)

class ChatEngine:
    model = ""
    temp = 0

    def __init__(self, model: str = "gpt-3.5-turbo", temp: float = 0):
        self.model = model
        self.temp = temp

@dataclass
class Message:
    role: str
    content: str

class TermChat:
    messages: List[Message] = []
    engine: ChatEngine

    def __init__(self, engine: ChatEngine, system_message: str, user_messages: List[str] = []):
        self.engine = engine
        self.messages.append(Message("system", system_message))
        self.messages.extend([Message("user", message) for message in user_messages])

    def get_answer(self, user_message: str = "") -> str:
        if user_message != "":
            self.messages.append(Message("user", user_message))

        chat_messages = [{"role": message.role, "content": message.content} for message in self.messages]
        response = openai.chat.completions.create(
            model=self.engine.model,
            messages=chat_messages,
            temperature=self.engine.temp,
        )
        answer = response.choices[0].message["content"]
        self.messages.append({"role": "assistant", "content": answer})
        return answer

    def clear_messages(self):
        first_message = self.messages[0]
        self.messages = []
        self.messages.append(first_message)


TermChat(ChatEngine(), "").get_answer()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
continue_term_prompt = """You are a terms helper robot \
your task is to guess, based on the list of terms, \
which one term, not in the list, \
is the most relevant to already given terms. Answer with one term please."""

def predict_next_term():
    engine = ChatEngine(temp=0.5)
    chat = Chat(engine, continue_term_prompt)
    terms_str = get_terms_string()
    answer = chat.get_answer(terms_str)
    print_lines(answer)
    chat.clear_messages()


predict_next_term()

In [ ]:
give_definition_prompt = """
You are a terms definition robot \
your task is to give a best possible definition to the user requested term. \

You should give a definition step by step, according to the this rules: \

1. Definition should contain only information about what the term intrinsically is, \
without any information about what the term is not, and what the term is related to.
Every entity that can be removed from definition without changing the meaning, should be removed.
2. Definition should based on the existing list of terms (list would be provided to you).
3. Check if there exist terms that also fit this definition and are not synonymous to the term.
4. Definition should be as short as possible. \

List of existing terms: '''{0}''' \

User requested definition for term: '''{1}''' \

Format of answer should be step by step thoughts: \
Raw term definition: <try to define user term for the first time. format 'term - definition'> <new line> \
Thoughts about definition: <your thoughts on how this term should be redefined according to rules> <new line> \

First retry: <try to define user term for the second time. format 'term - definition'> <new line> \
Thoughts about definition: <your thoughts on how this term should be redefined according to rules> <new line> \

Final try: <try to define user term for the third time. format 'term - definition'> <new line> \
"""

def give_definition(new_term):
    terms_str = get_terms_string()

    engine = ChatEngine(temp=0.2)
    chat = Chat(engine, give_definition_prompt.format(terms_str, new_term))

    answer = chat.get_answer()
    print(answer)
    chat.clear_messages()

give_definition("Up quark")

In [13]:
def add_term(term, definition, area=""):
    globalJson = load_global_static_data()

    termDef = '{0} - {1}'.format(term, definition)
    if area != "":
        globalJson['terms'].append({"area": area, "termDef": termDef})
    else:
        globalJson['terms'].append({"termDef": termDef})

    with open(global_path, "w") as f:
        json.dump(globalJson, f, indent=4)

add_term("Symmetry", f"{{system}} {{property}} that remains unchanged after a certain {{transformation}}", "phys")